## trying to read NOAA's SPC deadly tornado data<br>
it's written in these weird txt files and seems to be formatted as a table but isn't very readable<br>
noticed that it has headings with dashes and maybe each column has a distinct number of characters I can use to parse as a table?<br>
if I can figure this out, I can loop through the files [here](https://www.spc.noaa.gov/climo/torn/), rather than using excel's 'fixed width' feature

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import requests
import json
from bs4 import BeautifulSoup
import os

/Users/almatthews/.local/share/virtualenvs/notebooks-1rsMepG2/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/q0/1clc4jf97cxf003x4xgh89680000gr/T/ipykernel_62447/2917847026.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate 

In [3]:
today = pd.Timestamp("today").strftime("%m/%d/%Y")

In [4]:
from datawrapper import Datawrapper

In [5]:
dw_token = os.environ.get("dw_api")
dw = Datawrapper(access_token=dw_token)

In [6]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

---

In [7]:
# data=pandas.read_table('filename.txt', delimiter = ' ')
# data=pandas.read_fwf('filename.txt')

pd.read_fwf(
    "https://www.spc.noaa.gov/climo/torn/STAMTS21.txt",
    skiprows=9,
    nrows=1,
    delimiter="\n",
).iloc[0].to_list()

['    PREL  ACT   ACT  ACT  ACT    AV   21 20 19 18 AV  21 20 19 18 AV']

### In each table there's a bunch of dashes between the columns headers and the data
### it corresponds to the number of characters in each column
### so use the spaces between each set of dashes as 'fixed width' for the columns 

In [8]:
frames = []
frames.append(
    pd.read_fwf(
        "https://www.spc.noaa.gov/climo/torn/STAMTS.txt",
        skiprows=11,
        nrows=12,
        delimiter="\n",
    )
)
for n in range(14, 23):
    frames.append(
        pd.read_fwf(
            f"https://www.spc.noaa.gov/climo/torn/STAMTS{n}.txt",
            skiprows=11,
            nrows=12,
            delimiter="\n",
        )
    )

In [9]:
import re

In [10]:
clean_frames = []
for f in frames:
    space_list = [m.start() for m in re.finditer(" ", f.columns[0])]
    f["month"] = f[f.columns[0]].str[: space_list[0]]
    for n in range(1, len(space_list)):
        f["col" + str(n)] = f[f.columns[0]].str[space_list[n - 1] : space_list[n]]
    clean_frames.append(f[f.columns[1:]])

#### there are a bunch of extra columns, probably not worth batch dropping unless i can figure out a consistent pattern<br>
#### but at least now we have a semi-clean list of dataframes that we can work with as needed

In [19]:
# assign the years
df_latest = clean_frames[0]
# dfnames=[df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022]

# for n,y in zip(range(1,10),dfnames):
#    y=clean_frames[n]

In [28]:
labeled_frames = []
for f, y in zip(clean_frames[1:], range(2014, 2023)):
    labeled_frames.append(f.rename(columns=({f.columns[1]: "year_" + str(y)})))